In [ ]:
import os

# ---------------------------------------------------------
# Lista de VIDEOS para descargar (cambia SOLO los links)
# ---------------------------------------------------------
video_urls = [
    "https://www.youtube.com/watch?v=Ek6BE-Acsw8",
    "https://www.youtube.com/watch?v=SKeBZpjWTko",
    "https://www.youtube.com/watch?v=JMraBJsA9oI",
    "https://www.youtube.com/watch?v=IAG3OxPOUp0",
    "https://www.youtube.com/watch?v=xVsT21Rnyxk",
    "https://www.youtube.com/watch?v=vXSSgzXJy1Q"
]

# Carpeta donde se guardarán los videos
output_folder = "../videos"
os.makedirs(output_folder, exist_ok=True)

# ---------------------------------------------------------
# DESCARGA AUTOMÁTICA USANDO yt-dlp
# ---------------------------------------------------------
for url in video_urls:
    command = f'yt-dlp -f "bv[height<=720]+ba/best" -o "{output_folder}/%(title)s.%(ext)s" "{url}"'
    print(f"\n Descargando: {url}")
    os.system(command)

print("\n Descarga completada. Los videos están en la carpeta 'videos/'")



📥 Descargando: https://www.youtube.com/watch?v=Ek6BE-Acsw8

📥 Descargando: https://www.youtube.com/watch?v=SKeBZpjWTko

📥 Descargando: https://www.youtube.com/watch?v=JMraBJsA9oI

📥 Descargando: https://www.youtube.com/watch?v=IAG3OxPOUp0

📥 Descargando: https://www.youtube.com/watch?v=xVsT21Rnyxk

📥 Descargando: https://www.youtube.com/watch?v=vXSSgzXJy1Q

✅ Descarga completada. Los videos están en la carpeta 'videos/'


In [ ]:
import os
import subprocess

# ---------------------------------------------------------
# 1. LISTA DE VIDEOS A DESCARGAR (solo cambia los links)
# ---------------------------------------------------------
videos = [
    {"url": "https://www.youtube.com/watch?v=Ek6BE-Acsw8", "start": "00:00:02", "end": "00:00:05"},
    {"url": "https://www.youtube.com/watch?v=SKeBZpjWTko", "start": "00:00:01", "end": "00:00:04"},
    {"url": "https://www.youtube.com/watch?v=JMraBJsA9oI", "start": "00:00:03", "end": "00:00:08"},
    {"url": "https://www.youtube.com/watch?v=IAG3OxPOUp0", "start": "00:00:00", "end": "00:00:06"},
    {"url": "https://www.youtube.com/watch?v=xVsT21Rnyxk", "start": "00:00:02", "end": "00:00:05"},
    {"url": "https://www.youtube.com/watch?v=vXSSgzXJy1Q", "start": "00:00:01", "end": "00:00:05"}
]

# ---------------------------------------------------------
# 2. CARPETAS DE SALIDA
# ---------------------------------------------------------
raw_dir = "../videos"
trimmed_dir = "../videos_proc"

os.makedirs(raw_dir, exist_ok=True)
os.makedirs(trimmed_dir, exist_ok=True)

# ---------------------------------------------------------
# 3. DESCARGAR VIDEOS + RECORTARLOS
# ---------------------------------------------------------
print("\n🚀 Iniciando descarga y recorte...\n")

for item in videos:
    url = item["url"]
    start = item["start"]
    end = item["end"]

    print(f" Descargando: {url}")

    # Comando para descargar video
    download_cmd = (
        f'yt-dlp -f "bv[height<=720]+ba/best" '
        f'-o "{raw_dir}/%(title)s.%(ext)s" "{url}"'
    )

    os.system(download_cmd)

    # Encontrar el archivo descargado
    files = os.listdir(raw_dir)
    newest = max([f for f in files], key=lambda x: os.path.getctime(os.path.join(raw_dir, x)))
    input_path = os.path.join(raw_dir, newest)

    # Salida del video recortado
    output_path = os.path.join(trimmed_dir, f"trimmed_{newest}")

    print(f"  Recortando {newest} ...")

    # Comando FFmpeg para recortar
    trim_cmd = [
        "ffmpeg",
        "-i", input_path,
        "-ss", start,
        "-to", end,
        "-c", "copy",   # sin perder calidad
        output_path
    ]

    subprocess.run(trim_cmd, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

    print(f" Guardado: {output_path}\n")

print("\n Proceso completado: Videos descargados y recortados.")


In [ ]:
import os
from moviepy.editor import VideoFileClip

# -------------------------------------------------------------------
# CONFIGURACIÓN
# -------------------------------------------------------------------

INPUT_DIR  = "../videos"
OUTPUT_DIR = "../videos_proc"

videos_a_procesar = [
    {
        "filename": "Abecedario en Lengua de Señas colombianas - LSC.f136.mp4",
        "letter": "A",
        "cuts": [
            ("00:13", "00:15")
        ]
    },
    {
        "filename": "Abecedario en Lengua de Señas colombianas - LSC.f136.mp4",
        "letter": "B",
        "cuts": [
            ("00:16", "00:20")
        ]
    }
]

# -------------------------------------------------------------------
# UTILIDAD: convertir "MM:SS" → segundos
# -------------------------------------------------------------------

def convertir_a_segundos(t):
    """Convierte un tiempo 'MM:SS' o 'SS' a segundos enteros."""
    if isinstance(t, (int, float)):
        return t
    if isinstance(t, str) and ":" in t:
        minutos, segundos = t.split(":")
        return int(minutos) * 60 + int(segundos)
    return int(t)

# -------------------------------------------------------------------
# FUNCIÓN PARA OBTENER EL SIGUIENTE NÚMERO DE CLIP
# -------------------------------------------------------------------

def siguiente_indice_clip(folder):
    """
    Busca los clips existentes y determina el siguiente número.
    Ej: si hay clip_1.mp4, clip_2.mp4 → retorna 3
    """
    existentes = [
        f for f in os.listdir(folder)
        if f.startswith("clip_") and f.endswith(".mp4")
    ]

    if not existentes:
        return 1

    numeros = []
    for f in existentes:
        try:
            n = int(f.replace("clip_", "").replace(".mp4", ""))
            numeros.append(n)
        except:
            pass

    return max(numeros) + 1

# -------------------------------------------------------------------
# FUNCIÓN PARA RECORTAR VIDEOS
# -------------------------------------------------------------------

def recortar_video(input_path, output_folder, cuts):
    print(f"🔪 Procesando video: {input_path}")

    video = VideoFileClip(input_path)

    # Obtener primer índice disponible
    indice_clip = siguiente_indice_clip(output_folder)

    for start, end in cuts:
        start_s = convertir_a_segundos(start)
        end_s   = convertir_a_segundos(end)

        output_path = os.path.join(output_folder, f"clip_{indice_clip}.mp4")
        indice_clip += 1

        print(f"  ✂ Generando clip {output_path}: {start_s}s → {end_s}s")

        clip = video.subclip(start_s, end_s)
        clip.write_videofile(output_path, codec="libx264")

    video.close()

# -------------------------------------------------------------------
# MAIN
# -------------------------------------------------------------------

def main():
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    for item in videos_a_procesar:
        filename = item["filename"]
        letter   = item["letter"]
        cuts     = item["cuts"]

        print("\n============================")
        print(f"✏ Procesando letra: {letter}")
        print(f"Video: {filename}")

        input_path = os.path.join(INPUT_DIR, filename)

        # Carpeta por letra
        letter_folder = os.path.join(OUTPUT_DIR, letter)
        os.makedirs(letter_folder, exist_ok=True)

        recortar_video(input_path, letter_folder, cuts)

    print("\n PROCESO COMPLETADO")

if __name__ == "__main__":
    main()
